In [3]:
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
import keras.applications
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras import backend as K
import os
from sklearn.metrics import confusion_matrix
import numpy as np
import cv2

Using TensorFlow backend.


In [4]:
from google.colab import drive
# Dataset is stored at Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
img_shape = (150, 150, 3)
tb_log_dir = 'logs'
num_epochs = 2
num_classes = 6
train_directory = '/content/drive/My Drive/intel-image-classification/seg_train'
test_directory = '/content/drive/My Drive/intel-image-classification/seg_test'

In [0]:
def clear_folder(folder):
    if os.path.exists(folder):
        for the_file in os.listdir(folder):
            file_path = os.path.join(folder, the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                # elif os.path.isdir(file_path): shutil.rmtree(file_path)
            except Exception as e:
                print(e)

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


In [0]:
def build_model():

    basemodel = keras.applications.vgg16.VGG16(
        input_shape=img_shape,
        include_top=False,
        weights='imagenet',
        pooling='avg')

    for layer in basemodel.layers:
        layer.trainable = False

    basemodel.summary()
  
    model = Sequential()
    model.add(basemodel)
    '''
    model.add(Conv2D(32, kernel_size=(3, 3), input_shape =img_shape, padding='same', activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, kernel_size = (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(1, 1)))

    model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(1, 1)))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    '''
    model.add(Dense(4096, activation= 'relu'))
    model.add(Dropout(0.4))
    model.add(Dense(1028, activation= 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()

    return model

In [18]:
def load_images(directory, classes):    
    
    x = []
    y = []  

    
    for labels in os.listdir(directory): 

        label = classes[labels]
        print(labels)
        
        for image_file in os.listdir(directory+ '/' + labels): #Extracting the file name of the image from Class Label folder
            image = cv2.imread(directory+ '/' + labels+'/'+image_file) #Reading the image (OpenCV)
            image = cv2.resize(image,(150,150)) #Resize the image
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to grayscale
            #TODO: add image augmentation
            x.append(image)
            y.append(label)

    return x, y

classes = {2:'glacier', 4:'sea', 0:'buildings', 1:'forest', 5:'street', 3:'mountain'}
reverse_classes = {'glacier':2, 'sea':4, 'buildings':0, 'forest':1, 'street':5, 'mountain':3}

xtrain, ytrain = load_images(train_directory, reverse_classes)
np.save(train_directory+ '/trainxdata.npy', xtrain)
np.save(train_directory+ '/trainydata.npy', ytrain)
xtest, ytest = load_images(test_directory, reverse_classes)
np.save(test_directory+ '/testxdata.npy', xtest)
np.save(test_directory+ '/testydata.npy', ytest)


mountain
glacier
forest
street
sea
buildings
sea
mountain
street
glacier
forest
buildings


In [12]:
seed = 42
batch_size = 32

xtrain = np.load(train_directory+ '/trainxdata.npy') / 255
ytrain = np.load(train_directory+ '/trainydata.npy')
xtest = np.load(test_directory+ '/testxdata.npy') / 255
ytest = np.load(test_directory+ '/testydata.npy')

ytrain = keras.utils.to_categorical(ytrain)
ytest = keras.utils.to_categorical(ytest)

'''# different pipeline needed, because reading images from google drive is really really slow
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    data_format='channels_last',
    validation_split=0.25,
    #shear_range=0.2,
    #zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_data = train_datagen.flow_from_directory(
    train_directory,
    target_size=img_shape[:-1], # only height and width
    batch_size=batch_size,
    seed=seed,
    class_mode='categorical',
    shuffle=True,
    color_mode='rgb')

test_data = test_datagen.flow_from_directory(
    test_directory,
    target_size=img_shape[:-1], # only height and width
    class_mode='categorical',
    color_mode='rgb')

validation_data = train_datagen.flow_from_directory(
    train_directory, # same directory as training data
    target_size=img_shape[:-1],
    class_mode='categorical',
    batch_size=batch_size,
    subset='validation') # set as validation data'''



clear_folder(tb_log_dir)
tensorboard = TensorBoard(log_dir=tb_log_dir)
checkpoint = ModelCheckpoint(train_directory+ 'weights.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

model = build_model()
optimizer = optimizers.RMSprop(lr=1e-4)
# Labels are one hot encoded
model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics=["accuracy"])
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
print(model.summary)

model.fit(xtrain, ytrain, epochs=15, batch_size=32, validation_split = 0.25, callbacks=[tensorboard, checkpoint])
prediction = model.predict(xtest)


#class_labels = list(reverse_classes.keys())   

#confusion = metrics.classification_report(np.argmax(ytest, axis = 1), prediction, target_names=class_labels)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:707: RuntimeWarning: Can save best model only with val_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


10542/10542 [==============================] - 34s 3ms/step - loss: 0.5557 - acc: 0.8604 - val_loss: 9.5382 - val_acc: 0.3372
Epoch 3/15
10542/10542 [==============================] - 34s 3ms/step - loss: 0.3943 - acc: 0.8815 - val_loss: 9.7032 - val_acc: 0.3261
Epoch 4/15
10542/10542 [==============================] - 34s 3ms/step - loss: 0.3348 - acc: 0.8930 - val_loss: 9.8843 - val_acc: 0.3381
Epoch 5/15
 8192/10542 [======================>.......] - ETA: 5s - loss: 0.3135 - acc: 0.9016

KeyboardInterrupt: ignored